
## 1. Predicting housing prices using Regression
Simple example of supervised learning using decision trees and random forests.  From "Introduction to Machine Learning"

In [111]:
import pandas as pd
import numpy as np
from pandas import DataFrame as df
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [25]:
home_file_path = '~/Documents/ML_course/train.csv'
home_data = pd.read_csv(home_file_path) 
home_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


1. Predicting values using DecisionTreeRegressor, model accuracy using mean_absolute_error

In [73]:
y = home_data.SalePrice
feature_columns = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = home_data[feature_columns]

# Define model
home_model = DecisionTreeRegressor()
home_model.fit(X, y)
mean_value = df.mean(home_data.SalePrice)
print("relative error (in sample, no split) (%)", mean_absolute_error(y, home_model.predict(X))/mean_value*100)

relative error (in sample, no split) (%) 0.03446491583955324


Models' practical value $\rightarrow$ making predictions on $\textbf{new}$ data, so:

2. We exclude some data from the model-building process, then use those to test it - dividing the data in train data and test data.


In [74]:
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state = 0)
# Define model
home_model = DecisionTreeRegressor()
# Fit model
home_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = home_model.predict(test_X)                 
print("relative error (%)", mean_absolute_error(test_y, val_predictions)/mean_value*100)

relative error (%) 18.56042023230685


The error of almost 20% makes this model unusable. Since the "in sample error" is very small, we conclude that our model is overfit.
To improve this movel, we can experiment with:
* choosing different features
* different model types.

First approach:

3. Finding a sweet spot between over/underfitting by controlling the tree depth - number of nodes.

In [110]:
##function to compare Mean Absolute Error (MAE) scores for different numbers of nodes
def get_mae(max_leaf_nodes, train_X, test_X, train_y, test_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(test_X)
    mae = mean_absolute_error(test_y, preds_val)
    return(mae)

def best_nodes():
    for leaf_size in np.arange(2,200,50):
        scores = {leaf_size: get_mae(leaf_size, train_X, test_X, train_y, test_y)}
    return min(scores, key=scores.get)


# Fit the model with best_tree_size. Fill in argument to make optimal size
final_model = DecisionTreeRegressor(max_leaf_nodes=best_nodes(), random_state=1)

# fit the final model
final_model.fit(X, y)
val_predictions = final_model.predict(test_X)                 
print("relative error (%)", mean_absolute_error(test_y, val_predictions)/mean_value*100)
#best_nodes(max_leaf_nodes, train_X, test_X, train_y, test_y)

relative error (%) 7.838130878949268


4. We improved our model by 10% just adjusting the amount of nodes! Now using RandomForestRegressor instead of decision trees, we actually didn't find a better result:


In [116]:
forest_model = RandomForestRegressor(random_state=2)
forest_model.fit(train_X, train_y)
home_preds = forest_model.predict(test_X)
print("relative error (%)",mean_absolute_error(test_y, home_preds)/mean_value*100)

relative error (%) 12.679975565349173
